In [1]:
%load_ext autoreload
%autoreload 2

import meshio
import numpy as np
import torch
import trimesh

from fem import fea, eval
from graphs import build_graph
from utils import msh_to_trimesh, visualize

# Example Graph Generation

In [52]:
stl = "meshes/cantilever.stl"
msh = "meshes/cantilever.msh"
mesh = meshio.read(msh)


f = np.array([[10.0, 0.0, 0.0], [5.0, 0.0, 0.0]])
points = np.array([[0.0, 9.0, 30.0], [0.0, 9.0, 20.0]])
contacts = list(zip(points, f))

domain, stresses_vm = fea(
    contacts, contact_radius=2.0, debug=False, filename_stl=stl, filename_msh=msh
)
y = eval(domain, stresses_vm, mesh.points)
graph = build_graph(mesh, y, contacts)
print("Graph is valid?", graph.validate(raise_on_error=True))


Info    : Reading 'meshes/cantilever.msh'...
Info    : 27 entities
Info    : 4923 nodes
Info    : 26950 elements
Info    : Done reading 'meshes/cantilever.msh'
Graph is valid? True


## Visualize Graphs

In [53]:
mesh = meshio.read(msh)
mesh = msh_to_trimesh(mesh)
visualize(mesh, graph, jupyter_backend="html", force_arrows=True)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

## Load Graphs and Visualize

In [61]:
graphs = torch.load("data/cantilever_2_1.pt", weights_only=False)
visualize(mesh, graphs[0], jupyter_backend="html", force_arrows=True)

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [9]:
p1 = torch.tensor([[1.0, 2.0, 3.0], [0.0, 1.0, 2.0]])
p2 = torch.tensor([[4.0, 5.0, 6.0], [1.0, 2.0, 3.0]])
p = [p1, p2]

f1 = torch.tensor([[0.1, 0.2, 0.3], [0.0, 0.1, 0.2]])
f2 = torch.tensor([[0.4, 0.5, 0.6], [0.1, 0.2, 0.3]])
f = [f1, f2]

torch.stack([torch.stack(p), torch.stack(f)], dim=2)

tensor([[[[1.0000, 2.0000, 3.0000],
          [0.1000, 0.2000, 0.3000]],

         [[0.0000, 1.0000, 2.0000],
          [0.0000, 0.1000, 0.2000]]],


        [[[4.0000, 5.0000, 6.0000],
          [0.4000, 0.5000, 0.6000]],

         [[1.0000, 2.0000, 3.0000],
          [0.1000, 0.2000, 0.3000]]]])

# Evenly-Distributed-Load Dataset Generation

In [ ]:
def make_dataset(
    mesh: meshio.Mesh,
    f: np.ndarray,
    points: np.ndarray,
    stl="cantilever.stl",
    msh="cantilever.msh",
):
    """Generate n graphs from given contact points on one given mesh."""
    graphs = []
    for p, f_vec in zip(points, f):
        contacts = [(p, f_vec)]

        domain, stresses_vm = fea(
            contacts,
            contact_radius=2.0,
            debug=False,
            filename_stl=stl,
            filename_msh=msh,
        )
        y = eval(domain, stresses_vm, mesh.points)
        graphs.append(build_graph(mesh, y, contacts))

    return graphs


stl = "meshes/cantilever.stl"
msh = "meshes/cantilever.msh"
mesh = meshio.read(msh)

points = np.array(
    [
        [0.0, 5.0, 10.0],
        [0.0, 5.0, 20.0],
        [0.0, 5.0, 30.0],
        [0.0, 5.0, 40.0],
    ]
)
f = np.tile([10.0, 0.0, 0.0], (len(points), 1))

graphs = make_dataset(mesh, f, points, stl=stl, msh=msh)
torch.save(graphs, "cantilever_1_4.pt")